In [35]:
import pandas as pd
import numpy as np
import xgboost as xgb

#conda install -c conda-forge cufflinks-py
#conda install plotly
import ipywidgets as wg
from IPython.display import display

import cufflinks as cf
import chart_studio.plotly as py
import plotly.express as px

import matplotlib.pyplot as plt
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import inspect
import seaborn as sns

init_notebook_mode(connected=True)
cf.go_offline()
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, Normalizer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer, SimpleImputer
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.compose import ColumnTransformer
pd.options.display.max_columns = 200
pd.options.display.max_rows = 272

In [36]:
from sklearn.feature_selection import SelectKBest, VarianceThreshold

from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, explained_variance_score, mean_absolute_error, make_scorer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.decomposition import PCA

In [125]:
df = pd.read_csv("data/cleaned_df.csv", index_col="UnitID")

In [126]:
df.drop("Unnamed: 0", axis=1, inplace=True)

In [127]:
dependent_df = df[df.columns[-14:]]

In [128]:
dependent_df.head()

,Graduation rate total cohort (DRVGR2019),Graduation rate men (DRVGR2019),Graduation rate women (DRVGR2019),Graduation rate American Indian or Alaska Native (DRVGR2019),Graduation rate Asian/Native Hawaiian/Other Pacific Islander (DRVGR2019),Graduation rate Asian (DRVGR2019),Graduation rate Native Hawaiian or Other Pacific Islander (DRVGR2019),Graduation rate Black non-Hispanic (DRVGR2019),Graduation rate Hispanic (DRVGR2019),Graduation rate White non-Hispanic (DRVGR2019),Graduation rate two or more races (DRVGR2019),Graduation rate Race/ethnicity unknown (DRVGR2019),Graduation rate Nonresident alien (DRVGR2019),Transfer-out rate total cohort (DRVGR2019)
UnitID,,,,,,,,,,,,,,
180203,29.0,13.0,46.0,26.0,NaN,NaN,NaN,NaN,NaN,33.0,NaN,NaN,NaN,NaN
222178,61.0,56.0,65.0,25.0,58.0,58.0,NaN,53.0,50.0,67.0,54.0,0.0,57.0,NaN
138558,26.0,28.0,25.0,33.0,25.0,25.0,NaN,12.0,32.0,29.0,20.0,100.0,33.0,39.0
172866,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN
108232,45.0,38.0,51.0,67.0,56.0,57.0,40.0,20.0,33.0,50.0,40.0,28.0,57.0,14.0


In [129]:
features = df.iloc[:,:-14]

In [130]:
features.drop(["City location of institution (HD2019)", "Institution Name"], axis=1, inplace=True)

In [131]:
features = pd.get_dummies(features, drop_first=True)

In [132]:
features.head(2)

,Core_Revenues,Tuition_And_Fees,Government_Grants,Private_Gifts,Investment_Return,Sales_And_Services,Other_Revenues,Tuition_And_Fees_As_Dollar_Amount,Government_Grants_As_Dollar_Amount,Private_Gifts_As_Dollar_Amount,Investment_Return_As_Dollar_Amount,Sales_And_Services_As_Dollar_Amount,Other_Revenues_As_Dollar_Amount,Core_Expenses,Instruction_Expenses,Research_Expenses,Public_Service_Expenses,Academic_Support_Expenses,Student_Service_Expenses,Institutional_Support_Expenses,Other_Core_Expenses,Instruction_Expenses_As_Dollar_Amount,Research_Expenses_As_Dollar_Amount,Public_Service_Expenses_As_Dollar_Amount,Academic_Support_Expenses_As_Dollar_Amount,Student_Service_Expenses_As_Dollar_Amount,Institutional_Support_Expenses_As_Dollar_Amount,Other_Core_Expenses_As_Dollar_Amount,Grand total instructional_staff,Grand total men instructional_staff,Grand total women instructional_staff,American Indian or Alaska Native total instructional_staff,American Indian or Alaska Native men instructional_staff,American Indian or Alaska Native women instructional_staff,Asian total instructional_staff,Asian men instructional_staff,Asian women instructional_staff,Black or African American total instructional_staff,Black or African American men instructional_staff,Black or African American women instructional_staff,Hispanic or Latino total instructional_staff,Hispanic or Latino men instructional_staff,Hispanic or Latino women instructional_staff,Native Hawaiian or Other Pacific Islander total instructional_staff,Native Hawaiian or Other Pacific Islander men instructional_staff,Native Hawaiian or Other Pacific Islander women instructional_staff,White total instructional_staff,White men instructional_staff,White women instructional_staff,Two or more races total instructional_staff,Two or more races men instructional_staff,Two or more races women instructional_staff,Race/ethnicity unknown total instructional_staff,Race/ethnicity unknown men instructional_staff,Race/ethnicity unknown women instructional_staff,Nonresident alien total instructional_staff,Nonresident alien men instructional_staff,Nonresident alien women instructional_staff,Percent of full-time first-time undergraduates instructional_staff,Percent of full-time first-time undergraduates awarded federal state local instructional_staff,Grand total men instructional_staff_as_percentage,Grand total women instructional_staff_as_percentage,American Indian or Alaska Native total instructional_staff_as_percentage,American Indian or Alaska Native men instructional_staff_as_percentage,American Indian or Alaska Native women instructional_staff_as_percentage,Asian total instructional_staff_as_percentage,Asian men instructional_staff_as_percentage,Asian women instructional_staff_as_percentage,Black or African American total instructional_staff_as_percentage,Black or African American men instructional_staff_as_percentage,Black or African American women instructional_staff_as_percentage,Hispanic or Latino total instructional_staff_as_percentage,Hispanic or Latino men instructional_staff_as_percentage,Hispanic or Latino women instructional_staff_as_percentage,Native Hawaiian or Other Pacific Islander total instructional_staff_as_percentage,Native Hawaiian or Other Pacific Islander men instructional_staff_as_percentage,Native Hawaiian or Other Pacific Islander women instructional_staff_as_percentage,White total instructional_staff_as_percentage,White men instructional_staff_as_percentage,White women instructional_staff_as_percentage,Two or more races total instructional_staff_as_percentage,Two or more races men instructional_staff_as_percentage,Two or more races women instructional_staff_as_percentage,Race/ethnicity unknown total instructional_staff_as_percentage,Race/ethnicity unknown men instructional_staff_as_percentage,Race/ethnicity unknown women instructional_staff_as_percentage,Nonresident alien total instructional_staff_as_percentage,Nonresident alien men instructional_staff_as_percentage,Nonresident al

In [133]:
X = features.copy(deep=True)
y = dependent_df.copy(deep=True).iloc[:, 0]

In [134]:
pipeline = Pipeline(
    [
        ("imputer", KNNImputer()),
        ("scaler", StandardScaler()),
        ("classifier", ElasticNet(alpha=0.1, l1_ratio=0.5)),
    ]
)

In [135]:
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=0)

In [136]:
pipeline.fit(x_train, y_train)

Pipeline(steps=[('imputer', KNNImputer()), ('scaler', StandardScaler()),
                ('classifier', ElasticNet(alpha=0.1))])

In [137]:
y_pred = pipeline.predict(x_test)

In [138]:
mean_absolute_error(y_pred, y_test)

10.091106265945296

In [139]:
y_pred[:5]

array([29.76759151, 77.8818633 , 33.71305235, 25.90834104, 61.48164046])

In [140]:
y_test[:5]

UnitID
218919    41.0
212106    81.0
235431    33.0
236258    46.0
212115    52.0
Name: Graduation rate  total cohort (DRVGR2019), dtype: float64

# Baseline Elastic Net Model Predicts within 10%

## Lets do some feature engineering and model optimization to see if we can bring that down

In [141]:
clean_pipeline = Pipeline(
    [
        ("imputer", KNNImputer()),
        ("scaler", StandardScaler()),
    ]
)

In [142]:
new_features = clean_pipeline.fit_transform(features)

In [143]:
new_features = pd.DataFrame(new_features, columns=features.columns)

### First Step is to make principle components

In [144]:
# Create PCA Transformer
pca_transformer = PCA(n_components=1) 

In [147]:
df.columns[2:8] # Revenues Distribution

Index(['Tuition_And_Fees', 'Government_Grants', 'Private_Gifts',
       'Investment_Return', 'Sales_And_Services', 'Other_Revenues'],
      dtype='object')

In [148]:
#Transform these columns into a principle component
rev_dist_pc = pca_transformer.fit_transform(new_features[df.columns[2:8]])

In [149]:
dependent_df[[dependent_df.columns[0]]]

,Graduation rate total cohort (DRVGR2019)
UnitID,
180203,29.0
222178,61.0
138558,26.0
172866,0.0
108232,45.0
...,...
217059,63.0
141361,42.0
206695,41.0


In [152]:
new_features

,Core_Revenues,Tuition_And_Fees,Government_Grants,Private_Gifts,Investment_Return,Sales_And_Services,Other_Revenues,Tuition_And_Fees_As_Dollar_Amount,Government_Grants_As_Dollar_Amount,Private_Gifts_As_Dollar_Amount,Investment_Return_As_Dollar_Amount,Sales_And_Services_As_Dollar_Amount,Other_Revenues_As_Dollar_Amount,Core_Expenses,Instruction_Expenses,Research_Expenses,Public_Service_Expenses,Academic_Support_Expenses,Student_Service_Expenses,Institutional_Support_Expenses,Other_Core_Expenses,Instruction_Expenses_As_Dollar_Amount,Research_Expenses_As_Dollar_Amount,Public_Service_Expenses_As_Dollar_Amount,Academic_Support_Expenses_As_Dollar_Amount,Student_Service_Expenses_As_Dollar_Amount,Institutional_Support_Expenses_As_Dollar_Amount,Other_Core_Expenses_As_Dollar_Amount,Grand total instructional_staff,Grand total men instructional_staff,Grand total women instructional_staff,American Indian or Alaska Native total instructional_staff,American Indian or Alaska Native men instructional_staff,American Indian or Alaska Native women instructional_staff,Asian total instructional_staff,Asian men instructional_staff,Asian women instructional_staff,Black or African American total instructional_staff,Black or African American men instructional_staff,Black or African American women instructional_staff,Hispanic or Latino total instructional_staff,Hispanic or Latino men instructional_staff,Hispanic or Latino women instructional_staff,Native Hawaiian or Other Pacific Islander total instructional_staff,Native Hawaiian or Other Pacific Islander men instructional_staff,Native Hawaiian or Other Pacific Islander women instructional_staff,White total instructional_staff,White men instructional_staff,White women instructional_staff,Two or more races total instructional_staff,Two or more races men instructional_staff,Two or more races women instructional_staff,Race/ethnicity unknown total instructional_staff,Race/ethnicity unknown men instructional_staff,Race/ethnicity unknown women instructional_staff,Nonresident alien total instructional_staff,Nonresident alien men instructional_staff,Nonresident alien women instructional_staff,Percent of full-time first-time undergraduates instructional_staff,Percent of full-time first-time undergraduates awarded federal state local instructional_staff,Grand total men instructional_staff_as_percentage,Grand total women instructional_staff_as_percentage,American Indian or Alaska Native total instructional_staff_as_percentage,American Indian or Alaska Native men instructional_staff_as_percentage,American Indian or Alaska Native women instructional_staff_as_percentage,Asian total instructional_staff_as_percentage,Asian men instructional_staff_as_percentage,Asian women instructional_staff_as_percentage,Black or African American total instructional_staff_as_percentage,Black or African American men instructional_staff_as_percentage,Black or African American women instructional_staff_as_percentage,Hispanic or Latino total instructional_staff_as_percentage,Hispanic or Latino men instructional_staff_as_percentage,Hispanic or Latino women instructional_staff_as_percentage,Native Hawaiian or Other Pacific Islander total instructional_staff_as_percentage,Native Hawaiian or Other Pacific Islander men instructional_staff_as_percentage,Native Hawaiian or Other Pacific Islander women instructional_staff_as_percentage,White total instructional_staff_as_percentage,White men instructional_staff_as_percentage,White women instructional_staff_as_percentage,Two or more races total instructional_staff_as_percentage,Two or more races men instructional_staff_as_percentage,Two or more races women instructional_staff_as_percentage,Race/ethnicity unknown total instructional_staff_as_percentage,Race/ethnicity unknown men instructional_staff_as_percentage,Race/ethnicity unknown women instructional_staff_as_percentage,Nonresident alien total instructional_staff_as_percentage,Nonresident alien men instructional_staff_as_percentage,Nonresident al

In [153]:
data = {"rev_dist_pc":rev_dist_pc.flatten(), "target":dependent_df.iloc[:,0].to_numpy()}
df_pc1 = pd.DataFrame(data)

In [154]:
# Visualize principle component
fig = px.scatter(
    df_pc1, x="rev_dist_pc", y="target", height=600, width=800, trendline='ols',
    title="Revenue Distribution Principle Component", trendline_color_override="black",
    
)
results = px.get_trendline_results(fig) # Calculate R2 of Linear Regression
fig.show()

results.iloc[0]['px_fit_results'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     23.78
Date:                Sun, 05 Sep 2021   Prob (F-statistic):           1.15e-06
Time:                        19:01:58   Log-Likelihood:                -10416.
No. Observations:                2315   AIC:                         2.084e+04
Df Residuals:                    2313   BIC:                         2.085e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         51.5352      0.453    113.849      0.000      50.648      52.423
x1             1.6279      0.334      4.876      0.000       0.973       2.283
==============================================================================
Omnibus:                       24.868   Durbin-Watson:                   1.865
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               17.387
Skew:                          -0.088   Prob(JB):                     0.000168
Kurtosis:                       2.614   Cond. No.                         1.36
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [155]:
#polyfit principle component to graduation rate
df_pc1 = df_pc1.sort_values(by="rev_dist_pc")
x = df_pc1["rev_dist_pc"].to_numpy()
y = df_pc1["target"].to_numpy()

In [156]:
p1 = np.polyfit(x,y,2)

In [157]:
def f(a, m, b, x):
    return a * x**2 + m * x + b

In [158]:
df_pc1["rev_dist_squared"] = df_pc1["rev_dist_pc"].apply(lambda x: f(p1[0], p1[1], p1[2], x))

In [159]:
# Visualize new principle component
fig = px.scatter(
    df_pc1, x="rev_dist_squared", y="target", height=600, width=800, trendline='ols',
    title="Revenue Distribution Principle Component", trendline_color_override="black",
    
)
results = px.get_trendline_results(fig) # Calculate R2 of Linear Regression
fig.show()

results.iloc[0]['px_fit_results'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                  0.021
Method:                 Least Squares   F-statistic:                     50.50
Date:                Sun, 05 Sep 2021   Prob (F-statistic):           1.58e-12
Time:                        19:02:03   Log-Likelihood:                -10403.
No. Observations:                2315   AIC:                         2.081e+04
Df Residuals:                    2313   BIC:                         2.082e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -4.263e-14      7.266  -5.87e-15      1.000     -14.248      14.248
x1             1.0000      0.141      7.107      0.000       0.724       1.276
==============================================================================
Omnibus:                       10.367   Durbin-Watson:                   1.846
Prob(Omnibus):                  0.006   Jarque-Bera (JB):                8.094
Skew:                          -0.040   Prob(JB):                       0.0175
Kurtosis:                       2.722   Cond. No.                         834.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [119]:
df_pc1

,rev_dist_pc,target,rev_dist_squared
1171,-5.032599,16.0,26.533213
765,-4.875177,46.0,28.053418
1857,-4.843169,85.0,28.357098
1740,-4.832037,50.0,28.462280
1861,-4.787177,92.0,28.883912
...,...,...,...
728,2.552602,41.0,49.374290
5,2.553004,94.0,49.372773
234,2.686456,20.0,48.852932
185,5.222041,67.0,32.920992


In [ ]:
# calculate new R2 of polyfitted pc

In [61]:
df.columns[9:15] # Revenues Dollar Amount

Index(['Tuition_And_Fees_As_Dollar_Amount',
       'Government_Grants_As_Dollar_Amount', 'Private_Gifts_As_Dollar_Amount',
       'Investment_Return_As_Dollar_Amount',
       'Sales_And_Services_As_Dollar_Amount',
       'Other_Revenues_As_Dollar_Amount'],
      dtype='object')

In [65]:
df.columns[16:23] # expenses Distribution

Index(['Instruction_Expenses', 'Research_Expenses', 'Public_Service_Expenses',
       'Academic_Support_Expenses', 'Student_Service_Expenses',
       'Institutional_Support_Expenses', 'Other_Core_Expenses'],
      dtype='object')